**Recalculated Precision, Recall, and F1 Scores**

In [38]:
import pandas as pd

df = pd.read_csv('result.csv')

# Define a function to filter rows
def filter_rows(row):
    # Extract tuples from the lists
    
    predict_tags = row['predict_tag'][0]
    true_tags = row['true_tag'][0]

    # Check if the length of the tuples matches
    return len(predict_tags) <= len(true_tags)

# Apply the filter function to each row
mask = df.apply(filter_rows, axis=1)

# Filter the DataFrame using the boolean mask
filtered_df = df[mask]


print("Final Precision Score: ", filtered_df["precision"].mean())
print("Final Recall Score: ", filtered_df["recall"].mean())
print("Final F1 Score: ", filtered_df["f1"].mean())

Final Precision Score:  0.6453333333333334
Final Recall Score:  0.8714333333333332
Final F1 Score:  0.7115704761904763


In [42]:
import pandas as pd


# Function to calculate Average Precision (AP) for a single post
def average_precision(row):
    true_tags = row['true_tag'][2:-2].split("', '")  # Convert string to list
    predicted_tags = row['predict_tag'][2:-2].split("', '")  # Convert string to list
    
    # Initialize variables
    num_correct = 0
    precision_at_k = []
    relevant_so_far = 0
    
    # Calculate precision@k for each value of k
    for i, tag in enumerate(predicted_tags, start=1):
        if tag in true_tags:
            num_correct += 1
            precision_at_k.append(num_correct / i)
            relevant_so_far += 1
            
    # Calculate Average Precision (AP)
    if len(precision_at_k) > 0:
        return sum(precision_at_k) / min(len(true_tags), len(predicted_tags))  # Use the minimum of true and predicted tags
    else:
        return 0

# Apply the function to calculate AP for each row
df['average_precision'] = df.apply(average_precision, axis=1)

# Calculate Mean Average Precision (MAP)
map_score = df['average_precision'].mean()

print("Mean Average Precision (MAP):", map_score)


Mean Average Precision (MAP): 0.27479588888888884


**Hamming Loss**

In [31]:

from sklearn.metrics import hamming_loss
from sklearn.preprocessing import MultiLabelBinarizer
from ast import literal_eval

df = pd.read_csv("result.csv", converters={'predict_tag': literal_eval, 'true_tag': literal_eval}, encoding='ISO-8859-1')

pred_list=[]
true_list=[]

count = 0
for i, row in df.iterrows():
    count += 1
    pred = row['predict_tag']
    true = row['true_tag']
    if count%5 == 0:
        # print(list(true[0]))
        pred_list.append(list(pred[0]))
        true_list.append(list(true[0]))

# Binarize the labels
mlb = MultiLabelBinarizer()
y_true_binary = mlb.fit_transform(true_list)  # Fit on actual to cover all possible labels
y_pred_binary = mlb.transform(pred_list) 

hamming_score_avg = hamming_loss(y_true_binary, y_pred_binary)
print("Average Hamming Score:", hamming_score_avg)

for i in range(0,5):
    print("Predicted Tag:", pred_list[i])
    print("Actual Tag:", true_list[i])


Average Hamming Score: 0.03426315789473684
Predicted Tag: ['css', 'html', 'image', 'javascript', 'jquery']
Actual Tag: ['asp.net-mvc', 'css', 'html', 'javascript', 'jquery']
Predicted Tag: ['css', 'html', 'javascript', 'jquery', 'php']
Actual Tag: ['css', 'html', 'javascript', 'jquery']
Predicted Tag: ['android', 'eclipse', 'java', 'json', 'xml']
Actual Tag: ['android', 'java']
Predicted Tag: ['hibernate', 'java', 'mysql', 'spring', 'sql']
Actual Tag: ['hibernate', 'java', 'sql']
Predicted Tag: ['android', 'arrays', 'java', 'javascript', 'xml']
Actual Tag: ['android', 'java']


C:\Users\Lenovo\AppData\Roaming\Python\Python38\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['datetime'] will be ignored
  warnings.warn(
